In [ ]:
import ast
import cv2
import glob
import matplotlib.pyplot as plt  # Visualizzazione grafici e immagini
import numpy as np               # Operazioni numeriche su array multidimensionali
import os                       # Interazione con il sistema operativo (file, cartelle)
import pandas as pd             # Manipolazione e analisi di dati tabellari (DataFrame)
import random                   # Funzioni per generazione di numeri casuali
import sys                      # Funzioni per interagire con l'ambiente di esecuzione Python
import torch                    # Libreria principale di deep learning PyTorch (tensori, GPU)
import torch.nn as nn           # Costruzione modelli di reti neurali con layer predefiniti
import torch.nn.functional as F # Funzioni utili come attivazioni, loss, pooling

from collections import defaultdict, Counter
from fvcore.nn import FlopCountAnalysis  # Analisi di FLOPS e complessita' computazionale della rete
from IPython.display import display       # Visualizzazione ricca di output (notebook Jupyter)
from PIL import Image, ImageDraw, ImageFont   # Manipolazione e caricamento immagini
from torch.utils.data import Dataset, DataLoader  # Gestione dataset personalizzati e batching
from torchvision import transforms       # Trasformazioni e preprocessamento immagini
from torchvision.transforms import functional as F
from torchvision.transforms import functional as TF
from torchvision.transforms import Normalize
from torchvision.transforms.functional import resize


In [ ]:
# Detect CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
print(torch.version.cuda)  # Es: '12.1'
print(torch.cuda.get_device_name(0))

In [ ]:
# Percorso locale al dataset
dataset_path = 'CCPD2019' 

if os.path.exists(dataset_path):
    print("Dataset trovato in locale!")
else:
    print(f"Dataset non trovato al percorso: {dataset_path}")


In [ ]:
def print_directory_tree_jpg_png_limited(path, prefix="", is_last=True, max_image_files=5, is_root=False):
    """
    Stampa ricorsivamente la struttura ad albero di una directory, mostrando sottocartelle e file.
    
    Per i file immagine con estensione .jpg o .png limita la stampa ai primi `max_image_files` file,
    aggiungendo "..." se ce ne sono di piu'.
    
    Parametri:
    - path (str): percorso della directory da esplorare.
    - prefix (str): prefisso usato per l'indentazione (usato internamente nella ricorsione).
    - is_last (bool): indica se l'elemento corrente e' l'ultimo nella lista (per disegnare correttamente i connettori).
    - max_image_files (int): numero massimo di file immagine (.jpg, .png) da mostrare per ogni cartella.
    - is_root (bool): se True stampa solo il nome della cartella root senza prefisso.
    """

    connector = "└── " if is_last else "├── "

    if is_root:
        print(path)
    else:
        print(prefix + connector + os.path.basename(path))

    new_prefix = prefix + ("    " if is_last else "│   ")

    if os.path.isdir(path):
        items = sorted(os.listdir(path))
        full_paths = [os.path.join(path, item) for item in items]

        dirs = [item for item in full_paths if os.path.isdir(item)]
        files = [item for item in full_paths if os.path.isfile(item)]

        # Stampa tutte le sottocartelle
        for i, d in enumerate(dirs):
            is_last_dir = (i == len(dirs) - 1 and not files)
            print_directory_tree_jpg_png_limited(d, new_prefix, is_last_dir, max_image_files)

        # Se la cartella contiene immagini jpg o png, limita la stampa
        image_files = [f for f in files if f.lower().endswith((".jpg", ".png"))]
        if image_files:
            limited_images = image_files[:max_image_files]
            if len(image_files) > max_image_files:
                limited_images.append("...")

            for i, f in enumerate(limited_images):
                is_last_file = i == len(limited_images) - 1
                if f == "...":
                    print(new_prefix + "└── ...")
                else:
                    print(new_prefix + ("└── " if is_last_file else "├── ") + os.path.basename(f))
        else:
            # Altri file (non jpg o png): mostrali tutti
            for i, f in enumerate(files):
                is_last_file = i == len(files) - 1
                print(new_prefix + ("└── " if is_last_file else "├── ") + os.path.basename(f))

print("CCPD - Directory Tree:\n")
print_directory_tree_jpg_png_limited(dataset_path, is_root=True)


In [ ]:
# Funzione per leggere e mostrare le prime righe di un file txt
def show_txt_head(txt_file, num_lines=5):
    """
    Mostra le prime `num_lines` righe di un file di testo.

    Parametri:
    - txt_file (str): percorso del file di testo da leggere.
    - num_lines (int): numero di righe da visualizzare (default e' 5).

    La funzione apre il file in modalita' lettura, stampa riga per riga
    fino al numero specificato o fino alla fine del file.
    """
    print(f"Prime {num_lines} righe di {txt_file}:")
    with open(txt_file, 'r') as f:
        for _ in range(num_lines):
            line = f.readline().strip()
            if not line:
                break
            print(line)
    print()  # Riga vuota per separare le uscite

# Percorsi ai file txt di train, validation e test
train_txt_path = os.path.join(dataset_path, 'splits', 'train.txt')
val_txt_path = os.path.join(dataset_path, 'splits', 'val.txt')
test_txt_path = os.path.join(dataset_path, 'splits', 'test.txt')

# Mostra prime righe dei file train, val e test
show_txt_head(train_txt_path, num_lines=5)
show_txt_head(val_txt_path, num_lines=5)
show_txt_head(test_txt_path, num_lines=5)


In [ ]:
def show_first_existing_ccpd_sample(txt_file, dataset_path, title="Sample"):
    """
    Cerca e mostra la prima immagine elencata in un file di testo.

    Parametri:
    - txt_file (str): percorso al file di testo che contiene i percorsi relativi delle immagini.
    - dataset_path (str): percorso alla cartella radice del dataset.
    - title (str): titolo da visualizzare sopra l'immagine (default: "Sample").

    La funzione legge tutte le righe del file txt, costruisce i percorsi completi
    delle immagini unendo il dataset_path con i percorsi relativi, e cerca la prima immagine
    che esiste sul disco. Se la trova, la apre e la mostra con matplotlib.
    Se nessuna immagine valida viene trovata, stampa un messaggio di avviso.
    """
    # Legge tutte le righe dal file txt
    with open(txt_file, 'r') as f:
        lines = f.read().splitlines()
    
    # Cerca la prima immagine esistente
    for idx, img_rel_path in enumerate(lines):
        img_path = os.path.join(dataset_path, img_rel_path)
        if os.path.isfile(img_path):
            # Apre e mostra l'immagine
            img = Image.open(img_path)
            plt.figure(figsize=(8,6))
            plt.imshow(img)
            plt.title(f"{title} - Immagine indice {idx}")
            plt.axis('off')
            plt.show()
            return  # Esce dopo aver mostrato la prima immagine valida
    
    print(f"Nessuna immagine trovata valida in {txt_file}")


# Mostra la prima immagine valida da train, val e test
show_first_existing_ccpd_sample(train_txt_path, dataset_path, title="Training Sample")
show_first_existing_ccpd_sample(val_txt_path, dataset_path, title="Validation Sample")
show_first_existing_ccpd_sample(test_txt_path, dataset_path, title="Test Sample")


In [ ]:
# Analisi intero dataset (non eseguire)

def inspect_image(image_path):
    """
    Apre un'immagine e ne estrae informazioni chiave.

    Parametri:
    - image_path (str): percorso completo dell'immagine da analizzare.

    Restituisce:
    - dict con informazioni sull'immagine:
      - "mode": modalita' colore (es. 'RGB', 'L', ecc.)
      - "dtype": tipo di dato dei pixel (es. 'uint8')
      - "shape": dimensioni dell'immagine (altezza, larghezza, canali)
      - "min": valore minimo dei pixel
      - "max": valore massimo dei pixel
    - None se l'immagine non puo' essere aperta o e' corrotta.
    """
    try:
        img = Image.open(image_path)
        arr = np.array(img)
        return {
            "mode": img.mode,
            "dtype": str(arr.dtype),
            "shape": arr.shape,
            "min": arr.min(),
            "max": arr.max()
        }
    except Exception:
        # Gestione silenziosa degli errori
        return None


def inspect_txt_images(txt_path, data_path, name=""):
    """
    Analizza un insieme di immagini elencate in un file di testo e stampa statistiche aggregate.

    Parametri:
    - txt_path (str): percorso al file di testo contenente i percorsi relativi delle immagini.
    - data_path (str): percorso alla cartella radice del dataset.
    - name (str): nome descrittivo del dataset (es. 'Training Set') per stampa a video.

    Funzionamento:
    - Legge tutte le righe del file txt.
    - Per ogni immagine valida (leggibile), estrae informazioni tramite inspect_image.
    - Conta e stampa quante immagini sono analizzate e statistiche su modalita' colore,
      tipo dati e dimensioni delle immagini.
    - Ignora silenziosamente immagini non trovate o corrotte.
    """
    print(f"\n Analisi del dataset: {name}")
    
    # Legge tutte le righe (percorsi immagini)
    with open(txt_path, 'r') as f:
        lines = f.read().splitlines()

    stats = defaultdict(Counter)
    valid_count = 0

    for rel_path in lines:
        abs_path = os.path.join(data_path, rel_path)
        info = inspect_image(abs_path)

        if info is None:
            # Ignora silenziosamente immagini non trovate o corrotte
            continue

        valid_count += 1
        stats["mode"][info["mode"]] += 1
        stats["dtype"][info["dtype"]] += 1
        stats["shape"][str(info["shape"])] += 1

    print(f"Immagini analizzate: {valid_count} su {len(lines)}")
    print(f"\n Statistiche immagini per dataset '{name}':")

    print("Mode:", dict(stats["mode"]))
    print("Dtype:", dict(stats["dtype"]))
    print("Shape:", dict(stats["shape"]))


inspect_txt_images(train_txt_path, dataset_path, name="Training Set")
inspect_txt_images(val_txt_path, dataset_path, name="Validation Set")
inspect_txt_images(test_txt_path, dataset_path, name="Test Set")


In [ ]:
def decode_plate(plate_str):
    provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京",
                 "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏",
                 "陕", "甘", "青", "宁", "新", "警", "学", "O"]
    alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P',
                 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
    ads = alphabets + [str(d) for d in range(10)] + ['O']
    
    indices = list(map(int, plate_str.split('_')))
    prov = provinces[indices[0]] if indices[0] < len(provinces) else 'O'
    alpha = alphabets[indices[1]] if indices[1] < len(alphabets) else 'O'
    ads_str = ''.join(ads[i] if i < len(ads) else 'O' for i in indices[2:])
    return prov + alpha + ads_str

def parse_filename(filename):
    name = os.path.splitext(filename)[0]
    parts = name.split('-', maxsplit=6)
    if len(parts) != 7:
        raise ValueError(f"Filename non conforme: {filename}")
    
    bbox = None
    vertices = None

    try:
        leftup_str, rightdown_str = parts[2].split('_')
        x_min, y_min = map(int, leftup_str.split('&'))
        x_max, y_max = map(int, rightdown_str.split('&'))
        bbox = [x_min, y_min, x_max, y_max]
    except Exception:
        pass

    try:
        vertices = [list(map(int, v.split('&'))) for v in parts[3].split('_')]
    except Exception:
        pass

    plate_str = parts[4]
    annotation = decode_plate(plate_str)
    encoded_label = list(map(int, plate_str.split('_')))  # 🔥 nuovo formato usabile direttamente

    brightness = int(parts[5]) if parts[5].isdigit() else None
    blurriness = int(parts[6]) if parts[6].isdigit() else None

    return {
        'area': parts[0],
        'tilt': parts[1],
        'bbox': bbox,
        'vertices': vertices,
        'annotation': annotation,
        'encoded_label': encoded_label,
        'brightness': brightness,
        'blurriness': blurriness
    }

def create_csv_from_txt(txt_path, dataset_path, csv_output_path):
    with open(txt_path, 'r') as f:
        lines = f.read().splitlines()

    data = []
    for rel_path in lines:
        filename = os.path.basename(rel_path)
        try:
            parsed = parse_filename(filename)
            data.append({
                'image_path': rel_path,
                'area': parsed['area'],
                'tilt': parsed['tilt'],
                'bbox': parsed['bbox'],
                'vertices': parsed['vertices'],
                'annotation': parsed['annotation'],
                'encoded_label': parsed['encoded_label'],
                'brightness': parsed['brightness'],
                'blurriness': parsed['blurriness']
            })
        except Exception as e:
            print(f"Errore parsing {filename}: {e}")

    df = pd.DataFrame(data)
    df.to_csv(csv_output_path, index=False)
    print(f" CSV salvato: {csv_output_path}")


In [ ]:
create_csv_from_txt(train_txt_path, dataset_path, 'train.csv')
create_csv_from_txt(val_txt_path, dataset_path, 'val.csv')
create_csv_from_txt(test_txt_path, dataset_path, 'test.csv')


In [ ]:
def append_files_to_test(txt_path, source_path, category_name=None):
    """
    Aggiunge i file contenuti in source_path al file test.txt con prefisso 'category_name/'.
    
    Parametri:
    - txt_path: path al test.txt
    - source_path: cartella da cui leggere i file
    - category_name: nome prefisso (se None usa nome cartella finale)
    """
    if category_name is None:
        category_name = os.path.basename(os.path.normpath(source_path))

    files = [f for f in os.listdir(source_path) if os.path.isfile(os.path.join(source_path, f))]
    print(f" Trovati {len(files)} file in {source_path}")

    with open(txt_path, 'a') as f:
        for file_name in files:
            relative_path = f"{category_name}/{file_name}"
            f.write(relative_path + '\n')

    print(f" Aggiunti {len(files)} file a {txt_path}")

append_files_to_test('test.txt', '.\\CCPD2019\\ccpd_weather')


In [ ]:
test_txt_path = os.path.join(dataset_path, 'splits', 'test.txt')
create_csv_from_txt(test_txt_path, dataset_path, 'test.csv')

In [ ]:
def prepare_image_drawing(image_path):
    """
    Apre un'immagine e prepara un oggetto per disegnare su di essa.

    Parametri:
    - image_path (str): percorso dell'immagine da aprire.

    Restituisce:
    - img (PIL.Image): immagine aperta.
    - draw (ImageDraw.Draw): oggetto per disegnare sull'immagine.

    Se l'apertura fallisce, stampa un messaggio di errore e restituisce (None, None).
    """
    try:
        img = Image.open(image_path)
        draw = ImageDraw.Draw(img)
        return img, draw
    except Exception as e:
        print(f"Errore durante l'apertura dell'immagine {image_path}: {e}")
        return None, None

def draw_bbox(draw, bbox, outline='red', width=6):
    """
    Disegna una bounding box rettangolare sull'immagine.

    Parametri:
    - draw (ImageDraw.Draw): oggetto per disegno.
    - bbox (list o tuple): lista o tupla con coordinate [x_min, y_min, x_max, y_max].
    - outline (str): colore del bordo del rettangolo (default 'red').
    - width (int): spessore del bordo (default 6).

    Se bbox e' None, non viene disegnato nulla.
    """
    if bbox:
        draw.rectangle(bbox, outline=outline, width=width)

def draw_overlay_label(draw, annotation, bbox):
    """
    Disegna l'annotazione come testo sopra il bounding box con sfondo nero.

    Parametri:
    - draw (ImageDraw.Draw): oggetto per disegno.
    - annotation (str): testo da mostrare (es. targa).
    - bbox (list o tuple): coordinate del bounding box [x_min, y_min, x_max, y_max].

    Il testo viene disegnato in giallo con uno sfondo nero rettangolare
    posizionato subito sopra il bounding box.
    """
    if annotation and bbox:
        x_min, y_min, x_max, y_max = bbox

        # Font di default
        font = ImageFont.load_default()

        # Calcola dimensione del testo
        bbox_text = draw.textbbox((0, 0), annotation, font=font)
        text_width = bbox_text[2] - bbox_text[0]
        text_height = bbox_text[3] - bbox_text[1]


        # Coordinate del rettangolo di sfondo (leggermente più grande del testo)
        background_bbox = [x_min, y_min - text_height - 4, x_min + text_width + 4, y_min]

        # Disegna sfondo nero
        draw.rectangle(background_bbox, fill='black')

        # Disegna testo giallo sopra lo sfondo
        draw.text((x_min + 2, y_min - text_height - 2), annotation, fill='yellow', font=font)

def show_image_with_bbox_and_annotation(image_path, bbox, annotation):
    """
    Apre un'immagine, disegna bounding box e annotazione, e la mostra con matplotlib.

    Parametri:
    - image_path (str): percorso dell'immagine.
    - bbox (list o tuple): coordinate del bounding box.
    - annotation (str): testo dell'annotazione da disegnare.

    Se l'immagine non si apre, non mostra nulla.
    """
    img, draw = prepare_image_drawing(image_path)
    if img is None:
        return
    
    draw_bbox(draw, bbox, outline='red', width=6)
    draw_overlay_label(draw, annotation, bbox)
    
    plt.figure(figsize=(10,8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def show_two_examples_from_csv_using_bbox(csv_path, dataset_path, n=2):
    """
    Mostra i primi n esempi di immagini con bounding box e annotazione letti da un file CSV.

    Parametri:
    - csv_path (str): percorso del file CSV contenente i dati delle immagini.
    - dataset_path (str): percorso della cartella radice del dataset.
    - n (int): numero di esempi da mostrare (default 2).

    Per ogni immagine nel CSV, apre l'immagine, estrae il bounding box e l'annotazione,
    e mostra l'immagine con i relativi disegni.
    """
    df = pd.read_csv(csv_path)
    
    for idx, row in df.head(n).iterrows():
        rel_path = row['image_path']
        abs_path = os.path.join(dataset_path, rel_path)
        
        try:
            bbox = ast.literal_eval(row['bbox']) if pd.notna(row['bbox']) else None
        except Exception:
            bbox = None
        
        annotation = row['annotation'] if 'annotation' in row else None
        
        print(f"Mostro immagine: {rel_path}")
        show_image_with_bbox_and_annotation(abs_path, bbox, annotation)

# Esempio di uso
csv_path = 'train.csv' 

show_two_examples_from_csv_using_bbox(csv_path, dataset_path, n=2)


In [ ]:
class YoloCCPDTransformCentered:
    def __init__(self, dataset_path, size=(640, 640)):
        """
        Inizializza la trasformazione.

        Parametri:
        - dataset_path (str): percorso della cartella radice del dataset.
        - size (tuple): dimensione di output (altezza, larghezza), default (640, 640).
        """
        self.dataset_path = dataset_path
        self.size = size  # (H, W)

    def __call__(self, row):
        """
        Applica la trasformazione a una singola riga del dataset.

        Passaggi:
        - Apre l'immagine dal percorso indicato.
        - Calcola il bounding box della targa e il suo centro.
        - Calcola il ritaglio quadrato massimo possibile centrato sulla targa,
          che non esca dai bordi dell'immagine.
        - Ritaglia l'immagine con questo quadrato.
        - Aggiorna le coordinate del bounding box relativamente al ritaglio.
        - Ridimensiona il crop all'output desiderato (self.size).
        - Normalizza la bounding box nel formato YOLO (x_center, y_center, width, height).
        
        Parametri:
        - row (pandas.Series o dict): riga del dataset con almeno i campi 'image_path' e 'bbox'.

        Restituisce:
        - dict con:
            - 'image': tensore PyTorch dell'immagine ritagliata e ridimensionata.
            - 'label': tensore PyTorch con la label [classe, x_center, y_center, width, height].
        - None se l'immagine non esiste.
        """

        image_path = os.path.join(self.dataset_path, row['image_path'])
        if not os.path.isfile(image_path):
            return None

        image = Image.open(image_path).convert("RGB")
        orig_w, orig_h = image.size

        bbox = row['bbox']
        if isinstance(bbox, str):
            bbox = ast.literal_eval(bbox)

        x_min, y_min, x_max, y_max = bbox
        box_w = x_max - x_min
        box_h = y_max - y_min
        box_cx = (x_min + x_max) / 2
        box_cy = (y_min + y_max) / 2

        # Calcola il ritaglio quadrato massimo possibile centrato sulla targa
        dist_x = min(box_cx, orig_w - box_cx)
        dist_y = min(box_cy, orig_h - box_cy)
        half_side = min(dist_x, dist_y)

        crop_xmin = int(box_cx - half_side)
        crop_ymin = int(box_cy - half_side)
        crop_xmax = int(box_cx + half_side)
        crop_ymax = int(box_cy + half_side)

        crop = image.crop((crop_xmin, crop_ymin, crop_xmax, crop_ymax))

        # Aggiorna bbox relativa al crop
        new_x_min = x_min - crop_xmin
        new_y_min = y_min - crop_ymin
        new_x_max = x_max - crop_xmin
        new_y_max = y_max - crop_ymin

        # Converti crop in tensore e ridimensiona
        crop_tensor = TF.to_tensor(crop)
        crop_h, crop_w = crop_tensor.shape[1:]
        resized = TF.resize(crop_tensor, self.size, interpolation=Image.BILINEAR)

        scale_x = self.size[1] / crop_w
        scale_y = self.size[0] / crop_h

        # Ridimensiona bbox
        x_min_resized = new_x_min * scale_x
        y_min_resized = new_y_min * scale_y
        x_max_resized = new_x_max * scale_x
        y_max_resized = new_y_max * scale_y

        # Normalizza bbox (formato YOLO)
        x_center = (x_min_resized + x_max_resized) / 2 / self.size[1]
        y_center = (y_min_resized + y_max_resized) / 2 / self.size[0]
        width = (x_max_resized - x_min_resized) / self.size[1]
        height = (y_max_resized - y_min_resized) / self.size[0]

        # Clipping di sicurezza
        x_center = min(max(x_center, 0.0), 1.0)
        y_center = min(max(y_center, 0.0), 1.0)
        width = min(max(width, 0.0), 1.0)
        height = min(max(height, 0.0), 1.0)
        

        label = [0, x_center, y_center, width, height]

        return {
            'image': resized,
            'label': torch.tensor(label, dtype=torch.float32)
        }


In [ ]:
def plot_ccpd_pair(df, dataset_path, transformer=None, title='Sample'):
    """
    Visualizza un confronto tra l'immagine originale con bounding box e la sua versione trasformata.

    Parametri:
    - df (pandas.DataFrame): DataFrame contenente almeno le colonne 'image_path' e 'bbox'.
    - dataset_path (str): percorso della cartella radice dove sono salvate le immagini.
    - transformer (callable, opzionale): funzione o oggetto che trasforma una riga del DataFrame
      e restituisce un dizionario con 'image' (tensore trasformato) e 'label' (bbox normalizzata).
    - title (str, opzionale): titolo da mostrare sopra la figura.

    Funzionamento:
    - Per la prima immagine valida nel DataFrame:
      - Carica e mostra l'immagine originale con la bounding box disegnato.
      - Se fornito, applica la trasformazione e mostra l'immagine trasformata con bounding box aggiornata.
    - La bounding box della trasformata è convertita da formato YOLO (normalizzato) a coordinate pixel.
    - Visualizza il confronto in due subplot affiancati.
    """

    for _, row in df.iterrows():
        image_path = os.path.join(dataset_path, row['image_path'])
        if not os.path.isfile(image_path):
            continue  # Skip silently if image missing

        # Load original image and draw bbox
        image = Image.open(image_path).convert("RGB")
        draw_orig = ImageDraw.Draw(image)

        bbox = row['bbox']
        if isinstance(bbox, str):
            bbox = ast.literal_eval(bbox)

        draw_orig.rectangle(bbox, outline='red', width=3)

        if transformer:
            sample = transformer(row)
            if sample is None:
                continue

            transformed_img = sample['image'].permute(1, 2, 0).numpy()
            label = sample['label'].tolist()
            _, x_center, y_center, width, height = label
            H, W = sample['image'].shape[1:]

            # Convert YOLO bbox back to pixel coords
            x_min = int((x_center - width / 2) * W)
            y_min = int((y_center - height / 2) * H)
            x_max = int((x_center + width / 2) * W)
            y_max = int((y_center + height / 2) * H)

            fig, axes = plt.subplots(1, 2, figsize=(12, 6))
            axes[0].imshow(image)
            axes[0].set_title("Original + BBox")

            axes[1].imshow(transformed_img)
            axes[1].add_patch(plt.Rectangle(
                (x_min, y_min), x_max - x_min, y_max - y_min,
                edgecolor='red', facecolor='none', linewidth=2
            ))
            axes[1].set_title("Transformed + BBox")

        else:
            fig, ax = plt.subplots(figsize=(6, 6))
            ax.imshow(image)
            ax.set_title("Original")

        if title:
            fig.suptitle(title, weight='bold')

        plt.tight_layout()
        plt.show()
        break  # Show only first valid example


In [ ]:
transformer = YoloCCPDTransformCentered(dataset_path=dataset_path)
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

# Mostra esempio dal TRAIN

plot_ccpd_pair(train_df, dataset_path, transformer=transformer, title="Train Sample")

# Mostra esempio dal VAL
plot_ccpd_pair(val_df, dataset_path, transformer=transformer, title="Validation Sample")

# Mostra esempio dal TEST
plot_ccpd_pair(test_df, dataset_path, transformer=transformer, title="Test Sample")


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import torch
from torchvision import transforms

def create_yolo_format(dataset_path, csv_paths, transformer, max_samples=None):
    """
    Crea dataset in formato YOLOv5 compatibile con cartelle e file corretti.

    Parametri:
    - dataset_path (str): cartella radice del dataset.
    - csv_paths (dict): dizionario con chiavi 'train', 'val', 'test' e valori i path ai csv corrispondenti.
    - transformer (callable): funzione che applica trasformazione YoloCCPDTransformCentered,
      riceve una riga DataFrame e restituisce dict {'image': tensore, 'label': tensore}.
    - max_samples (dict, opzionale): dizionario con chiavi 'train', 'val', 'test' e valori il numero massimo di
      immagini da processare per ciascuno (es. {'train': 2000, 'val': 500, 'test': 500}).

    Funzionamento:
    - Per ogni split, legge il csv e itera su ogni immagine fino a max_samples[split] se definito.
    - Applica trasformazione.
    - Converte l'immagine in PIL e la salva in 'YOLO_format/images/{split}/' solo se non già esistente.
    - Scrive il file label YOLO in 'YOLO_format/labels/{split}/' solo se non già esistente.
    """
    yolo_root = os.path.join(dataset_path, 'YOLO_format')
    os.makedirs(yolo_root, exist_ok=True)

    for split, csv_file in csv_paths.items():
        df = pd.read_csv(csv_file)

        split_img_dir = os.path.join(yolo_root, 'images', split)
        split_label_dir = os.path.join(yolo_root, 'labels', split)
        os.makedirs(split_img_dir, exist_ok=True)
        os.makedirs(split_label_dir, exist_ok=True)

        # Imposta il limite se definito
        if max_samples and split in max_samples:
            df = df.iloc[:max_samples[split]]

        # Progress bar
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {split}"):
            img_name = os.path.basename(row['image_path'])
            img_save_path = os.path.join(split_img_dir, img_name)
            label_filename = os.path.splitext(img_name)[0] + '.txt'
            label_save_path = os.path.join(split_label_dir, label_filename)

            # CONTROLLO: se entrambi esistono, salta
            if os.path.exists(img_save_path) and os.path.exists(label_save_path):
                continue

            sample = transformer(row)
            if sample is None:
                continue

            img_pil = transforms.ToPILImage()(sample['image'].cpu())
            img_pil.save(img_save_path)

            labels = sample['label'].cpu().numpy()

            # Se label ha più oggetti, scriviamo tutte le righe
            with open(label_save_path, 'w') as f:
                if labels.ndim == 1:
                    labels = labels.reshape(1, -1)

                for obj in labels:
                    class_id, x_center, y_center, width, height = obj
                    f.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    print(" Creazione dataset in formato YOLOv5 completata.")


In [ ]:
# Esempio di chiamata
transformer = YoloCCPDTransformCentered(dataset_path=dataset_path)
csv_paths = {'train': 'train.csv', 'val': 'val.csv', 'test': 'test.csv'}
create_yolo_format(dataset_path, csv_paths, transformer)

In [ ]:
# TEST SPLIT .CSV:
import pandas as pd

def split_test_csv_by_directory(test_csv_path, output_dir):
    categories = ['ccpd_blur', 'ccpd_challenge', 'ccpd_db', 'ccpd_fn',
                  'ccpd_rotate', 'ccpd_tilt', 'ccpd_weather']

    df = pd.read_csv(test_csv_path)
    os.makedirs(output_dir, exist_ok=True)

    for cat in categories:
        filtered_df = df[df['image_path'].str.split('/').str[0] == cat]
        csv_path = os.path.join(output_dir, f"{cat}.csv")
        filtered_df.to_csv(csv_path, index=False)
        print(f"Creato {csv_path} con {len(filtered_df)} immagini.")

split_test_csv_by_directory('test.csv',"./csv_splits")


In [ ]:
# Creazione dei dataset splittati per il test:
transformer = YoloCCPDTransformCentered(dataset_path=dataset_path)
csv_paths = {'ccpd_blur': './csv_splits/ccpd_blur.csv', 'ccpd_db': './csv_splits/ccpd_db.csv', 'ccpd_fn': './csv_splits/ccpd_fn.csv','ccpd_rotate': './csv_splits/ccpd_rotate.csv', 'ccpd_tilt': './csv_splits/ccpd_tilt.csv', 'ccpd_weather': './csv_splits/ccpd_weather.csv','ccpd_challenge':'./csv_splits/ccpd_challenge.csv'}
create_yolo_format(dataset_path, csv_paths, transformer)

In [ ]:
def check_and_visualize_yolo_dataset(yolo_root):
    """
    Controlla la consistenza del dataset YOLOv5 e visualizza un esempio per ogni split.

    Parametri:
    - yolo_root (str): percorso della cartella principale YOLO_format contenente
      'images/{split}/' e 'labels/{split}/'.

    Funzionamento:
    - Per ogni split ('train', 'val', 'test'):
        - Conta il numero totale di immagini (.jpg).
        - Verifica quante immagini hanno il file label (.txt) corrispondente.
        - Stampa queste informazioni.
        - Visualizza la prima immagine con label valida disegnando il bounding box.
    - Se non sono trovate immagini con label valide in uno split, lo segnala.
    """

    splits = ['train', 'val', 'test']

    for split in splits:
        img_dir = os.path.join(yolo_root, 'images', split)
        label_dir = os.path.join(yolo_root, 'labels', split)

        img_files = sorted(glob.glob(os.path.join(img_dir, '*.jpg')))
        total_imgs = len(img_files)
        valid_labels = 0
        example_img = None
        example_bbox = None

        for img_path in img_files:
            base_name = os.path.splitext(os.path.basename(img_path))[0]
            label_path = os.path.join(label_dir, base_name + '.txt')

            if os.path.isfile(label_path):
                valid_labels += 1

                # Prendi il primo esempio con label valida
                if example_img is None:
                    example_img = Image.open(img_path).convert('RGB')
                    with open(label_path, 'r') as f:
                        line = f.readline().strip()
                        if line:
                            parts = line.split()
                            x_center, y_center, width, height = map(float, parts[1:5])
                            w, h = example_img.size
                            x_min = int((x_center - width / 2) * w)
                            y_min = int((y_center - height / 2) * h)
                            x_max = int((x_center + width / 2) * w)
                            y_max = int((y_center + height / 2) * h)
                            example_bbox = [x_min, y_min, x_max, y_max]

        print(f"Split: {split}")
        print(f"Totale immagini: {total_imgs}")
        print(f"Immagini con label valide: {valid_labels}\n")

        if example_img is not None and example_bbox is not None:
            draw = ImageDraw.Draw(example_img)
            draw.rectangle(example_bbox, outline='red', width=3)

            plt.figure(figsize=(6,6))
            plt.imshow(example_img)
            plt.title(f"Esempio da {split} con bounding box")
            plt.axis('off')
            plt.show()
        else:
            print(f"Nessun esempio valido trovato per split {split}\n")


In [ ]:
yolo_root = os.path.join(dataset_path, 'YOLO_format')
check_and_visualize_yolo_dataset(yolo_root)

In [ ]:
%matplotlib inline

import sys
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
from torchvision import transforms

# --- Configurazione percorsi ---
repo_dir = 'yolov5'   # Modifica se serve
weights_path = os.path.join(repo_dir, 'yolov5s.pt')  # Verifica che esista
dataset_YOLO = 'CCPD2019/YOLO_format/images/train'  # Percorso immagini train

# Aggiungi repo yolov5 al path per importare moduli interni
sys.path.insert(0, repo_dir)

# Importa modello
from models.common import DetectMultiBackend

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Carica modello
model = DetectMultiBackend(weights_path, device=device, dnn=False)

# Funzione inferenza e visualizzazione con NMS corretta
def infer_and_plot(model, img_path, device, input_size=(640, 640), conf_thres=0.25, iou_thres=0.45):
    preprocess = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
    ])

    img = Image.open(img_path).convert('RGB')
    input_tensor = preprocess(img).unsqueeze(0).to(device)

    # Inference (output raw)
    pred = model(input_tensor)

    # Importa la funzione di non max suppression dalla repo yolov5
    from utils.general import non_max_suppression

    detections = non_max_suppression(pred, conf_thres, iou_thres)[0]

    print(f"Risultati per {os.path.basename(img_path)}:")
    if detections is None or len(detections) == 0:
        print("Nessuna predizione trovata.")
        return

    img_cv = cv2.imread(img_path)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

    for *xyxy, conf, cls in detections.cpu().numpy():
        x1, y1, x2, y2 = map(int, xyxy)
        label = f"{model.names[int(cls)]} {conf:.2f}"
        cv2.rectangle(img_cv, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img_cv, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    plt.figure(figsize=(10, 8))
    plt.imshow(img_cv)
    plt.axis('off')
    plt.show()

# Inferenza su prime 2 immagini
image_files = sorted(os.listdir(dataset_YOLO))[:2]

for img_file in image_files:
    img_path = os.path.join(dataset_YOLO, img_file)
    infer_and_plot(model, img_path, device)


In [ ]:
# Spostati nella cartella yolov5 ed esegui il training in un'unica riga
!cd yolov5 && python yolov5/train.py   --img 640   --batch 25   --epochs 10   --data data.yaml   --weights yolov5s.pt   --optimizer Adam   --hyp yolov5/data/hyps/hyp.scratch-low.yaml  --name yolov5_ccpd_adam

# comando per inference:

python yolov5/val.py --weights best_yolo_weights.pt --data data_test.yaml --task test --batch 25 --workers 0